In [1]:
import numpy as np
import sys
from pyinstrument import Profiler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import os
sys.path.append(os.path.dirname(os.getcwd()))
from bart_playground import *

import bartz

In [2]:
proposal_probs = {"grow" : 0.4,
                  "prune" : 0.4,
                  "change" : 0.1,
                  "swap" : 0.1}
generator = DataGenerator(n_samples=160, n_features=2, noise=0.1, random_seed=42)
X, y = generator.generate(scenario="piecewise_flat")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
np.set_printoptions(suppress=True)
print(y_train[:12])

[ 0.50327821  0.60672224  0.26898966  0.55211673  0.50693811  0.66162097
 -0.64127659  0.65112284  0.03487759  0.23276531  0.44055996  0.38216964]


In [3]:
profiler = Profiler()
profiler.start()
bart = ChangeNumTreeBART(ndpost=200, nskip=100, n_trees=200, proposal_probs=proposal_probs)
bart.fit(X_train, y_train)
profiler.stop()
profiler.print()

Iterations:   1%|          | 2/300 [00:00<00:45,  6.53it/s]

Iterations:   1%|▏         | 4/300 [00:00<00:49,  6.03it/s]

Iterations:   2%|▏         | 5/300 [00:00<00:54,  5.38it/s]

Iterations:   2%|▏         | 7/300 [00:01<01:06,  4.41it/s]

Iterations:   3%|▎         | 8/300 [00:01<01:18,  3.70it/s]

...

Iterations:  97%|█████████▋| 292/300 [00:51<00:00, 10.87it/s]

Iterations:  98%|█████████▊| 294/300 [00:51<00:00, 11.79it/s]

Iterations:  99%|█████████▉| 298/300 [00:52<00:00, 12.29it/s]

Iterations: 100%|██████████| 300/300 [00:52<00:00,  5.73it/s]



  _     ._   __/__   _ _  _  _ _/_   Recorded: 18:10:33  Samples:  51436
 /_//_/// /_\ / //_// / //_'/ //     Duration: 52.419    CPU time: 51.172
/   _/                      v5.0.1

Profile at C:\Windows\Temp\ipykernel_25160\4120808940.py:2

52.418 <module>  C:\Windows\Temp\ipykernel_25160\4120808940.py:1
└─ 52.418 ChangeNumTreeBART.fit  bart_playground\bart.py:22
   └─ 52.415 NTreeSampler.run  bart_playground\samplers.py:69
      └─ 52.075 NTreeSampler.one_iter  bart_playground\samplers.py:226
         ├─ 20.728 Swap.propose  bart_playground\moves.py:34
         │  ├─ 7.669 Swap.try_propose  bart_playground\moves.py:146
         │  │  ├─ 4.469 Tree.swap_split  bart_playground\params.py:296
         │  │  │  └─ 4.336 Tree.change_split  bart_playground\params.py:289
         │  │  │     └─ 4.239 Tree.update_n  bart_playground\params.py:384
         │  │  │        ├─ 2.176 Tree.update_n  bart_playground\params.py:384
         │  │  │        │  ├─ 0.951 sum  <__array_function__ internal

In [4]:
rf = RandomForestRegressor()
lr = LinearRegression()
rf.fit(X_train, y_train)
lr.fit(X_train, y_train)

btz = bartz.BART.gbart(np.transpose(X_train), y_train, ntree=100, ndpost=200, nskip=100)
btpred_all = btz.predict(np.transpose(X_test))
btpred = np.mean(np.array(btpred_all), axis=0)

Iteration 100/300 P_grow=0.55 P_prune=0.45 A_grow=0.36 A_prune=0.36 (burnin)
Iteration 200/300 P_grow=0.57 P_prune=0.43 A_grow=0.35 A_prune=0.37
Iteration 300/300 P_grow=0.57 P_prune=0.43 A_grow=0.39 A_prune=0.40


In [5]:
bart.predict(X_test)

TypeError: list indices must be integers or slices, not list

In [ ]:

models = {"bart" : bart, 
          "rf" : rf, 
          "lr" : lr,
          "btz" : btz}
results = {}
for model_name, model in models.items():
    if model_name == "btz":
        results[model_name] = mean_squared_error(y_test, btpred)
    else:
        results[model_name] = mean_squared_error(y_test, model.predict(X_test))
results